# Data Cleaning Pandas Assignment

In [95]:
# It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. 
# How do you tidy it up so you can get on with the analysis? 
# Take this monstrosity as the DataFrame to use in the following puzzles:
# df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 'Budapest_PaRis', 'Brussels_londOn'],
#                    'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
#                    'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
#                    'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
#                    '12. Air France', '"Swiss Air"']})

In [96]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', '12. Air France', '"Swiss Air"']})
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


1. Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column).

In [97]:
for i in range(len(df)):
    #print(df.at[i, 'FlightNumber'])
    if i>0:
        df.at[i,'FlightNumber'] = df.at[i-1, 'FlightNumber'] + 10
df = df.astype({'FlightNumber':int})
print(df)  

            From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris         10045      [23, 47]               KLM(!)
1      MAdrid_miLAN         10055            []    <Air France> (12)
2  londON_StockhOlm         10065  [24, 43, 87]  (British Airways. )
3    Budapest_PaRis         10075          [13]       12. Air France
4   Brussels_londOn         10085      [67, 32]          "Swiss Air"


2. The From_To column would be better as two separate columns! 
Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. 
Assign the correct column names to this temporary DataFrame.

In [98]:
tdf = df.copy()
tdf[['From','To']] = tdf.From_To.str.split("_", expand=True)
tdf

,From_To,FlightNumber,RecentDelays,Airline,From,To
0,LoNDon_paris,10045,"[23, 47]",KLM(!),LoNDon,paris
1,MAdrid_miLAN,10055,[],<Air France> (12),MAdrid,miLAN
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. ),londON,StockhOlm
3,Budapest_PaRis,10075,[13],12. Air France,Budapest,PaRis
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air""",Brussels,londOn


3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. 
Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [99]:
tdf['From'] = tdf['From'].str.capitalize()
tdf['To'] = tdf['To'].str.capitalize()
tdf

,From_To,FlightNumber,RecentDelays,Airline,From,To
0,LoNDon_paris,10045,"[23, 47]",KLM(!),London,Paris
1,MAdrid_miLAN,10055,[],<Air France> (12),Madrid,Milan
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,Budapest_PaRis,10075,[13],12. Air France,Budapest,Paris
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air""",Brussels,London


4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [100]:
df = df.combine_first(tdf)
df = df.drop(columns='From_To')
df.index

RangeIndex(start=0, stop=5, step=1)

5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN. Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [101]:
max_list_len = df['RecentDelays'].str.len().max()
cnlist = ["delay_" + str(i) for i in range(max_list_len)]
df[cnlist] = pd.DataFrame(df.RecentDelays.tolist(), index= df.index)
df = df.drop(columns='RecentDelays')
df

,Airline,FlightNumber,From,To,delay_0,delay_1,delay_2
0,KLM(!),10045,London,Paris,23.0,47.0,NaN
1,<Air France> (12),10055,Madrid,Milan,NaN,NaN,NaN
2,(British Airways. ),10065,London,Stockholm,24.0,43.0,87.0
3,12. Air France,10075,Budapest,Paris,13.0,NaN,NaN
4,"""Swiss Air""",10085,Brussels,London,67.0,32.0,NaN
